In [1]:
import pandas as pd
import matplotlib.pyplot as plt
def Markemotions(emotions):
    df1 = pd.DataFrame({'Netural':emotions['neutral']})
    df2 = pd.DataFrame({'sad':emotions['sad']})
    df3 = pd.DataFrame({'happy':emotions['happy']})
    df4 = pd.DataFrame({'angry':emotions['angry']})
    df5 = pd.DataFrame({'disgust':emotions['disgust']})
    df6 = pd.DataFrame({'surprise':emotions['surprise']})
    df7 = pd.DataFrame({'fear':emotions['fear']})
    df8= pd.concat([df1, df2,df3,df4,df5,df6,df7])
    df8.to_csv("Emotions.csv",index=False)
    print("df is saved")
def plot_emotions(emotion_dict):
    angry=[]
    disgust=[]
    fear=[]
    happy=[]
    sad=[]
    surprise=[]
    neutral=[]
    for i in range(len(emotion_dict.keys())):
        angry.append(list(emotion_dict.values())[i]['angry'])
        disgust.append(list(emotion_dict.values())[i]['disgust'])
        fear.append(list(emotion_dict.values())[i]['fear'])
        happy.append(list(emotion_dict.values())[i]['happy'])
        sad.append(list(emotion_dict.values())[i]['sad'])
        surprise.append(list(emotion_dict.values())[i]['surprise'])
        neutral.append(list(emotion_dict.values())[i]['neutral'])
    #plt.scatter(list(emotion_dict.keys()), angry, color='red', marker='o' ,alpha=0.2, cmap='viridis')
    #plt.scatter(list(emotion_dict.keys()), sad, color='blue', marker='o' ,alpha=0.2, cmap='viridis')

    fig = plt.figure(figsize=(8, 5), dpi=80)
    plt.bar(list(emotion_dict.keys()), neutral,color='teal',linestyle='dashed' , label='Neutral')
    plt.bar(list(emotion_dict.keys()), happy,color='m',linestyle='dashed' , label='Happy')
    plt.bar(list(emotion_dict.keys()), sad,color='blue',linestyle='dashed' , label='Sad')
    plt.bar(list(emotion_dict.keys()), angry,color='red',linestyle='dashed' , label='Angry')
    plt.bar(list(emotion_dict.keys()), disgust,color='g',linestyle='dashed' , label='Disgust')
    plt.bar(list(emotion_dict.keys()), fear,color='black',linestyle='dashed' , label='Fear')
    plt.bar(list(emotion_dict.keys()), surprise,color='y',linestyle='dashed' , label='Surprise')
    plt.legend(loc='upper right',fontsize=10)
    #plt
    plt.savefig('Emotion_plot.png')
    print("plot has been saved")

In [2]:
def plot_piechart(Commen_emotion):
    my_dict = {i:Commen_emotion.count(i) for i in Commen_emotion}

    colors = ['#99ff99','#ffcc99','#ff9999','blue','green','yellow','red','purple']
    plt.pie([float(my_dict[v]) for v in my_dict],colors=colors, labels=[str(k) for k in my_dict], autopct='%1.1f%%', startangle=90)
    plt.savefig('Pie_chart.png')

In [3]:
from PyQt5 import QtGui
from PyQt5.QtWidgets import QWidget, QApplication, QLabel, QVBoxLayout
from PyQt5.QtGui import QPixmap
import sys
import cv2
from PyQt5.QtCore import pyqtSignal, pyqtSlot, Qt, QThread
import numpy as np
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QHBoxLayout, QVBoxLayout, QLabel,QSlider, QStyle, QSizePolicy, QFileDialog
import sys
from PyQt5.QtMultimedia import QMediaPlayer, QMediaContent
from PyQt5.QtMultimediaWidgets import QVideoWidget
from PyQt5.QtGui import QIcon, QPalette
from PyQt5.QtCore import Qt, QUrl
import cv2
from deepface import DeepFace
from datetime import datetime,timedelta
from statistics import mode
import numpy as np
global movie_name
from PyQt5.QtGui import QPalette , QColor,QFont,QIcon ,QPixmap
class VideoThread(QThread):
    change_pixmap_signal = pyqtSignal(np.ndarray)

    def run(self):
        # capture from web cam
        facecascade=cv2.CascadeClassifier(cv2.data.haarcascades+ "haarcascade_frontalface_alt2.xml")
        font=cv2.FONT_HERSHEY_SIMPLEX
        

        frame_count=0
        emotion_dict={}
        Commen_emotion=[]

        #ret=True
        count_time=0
        time=0
        now=timedelta(seconds=time)
        time_list=[]

        empty_dict = {'neutral': [], 'happy': [],'sad': [], 'angry': [],'disgust': [], 'surprise': [], 'fear': []}
        cap = cv2.VideoCapture(movie_name)
        while True:
            ret, frame = cap.read()
            fps = cap.get(cv2.CAP_PROP_FPS)
            if ret:
                frame = cv2.GaussianBlur(frame,(3,3),1)
                gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
                faces=facecascade.detectMultiScale(gray,1.5,4)
                dominant_emotion=""
                dominant_emotion_Percent=0
                frame_count=frame_count+1
                time = float(frame_count)/fps
                for (x,y,w,h) in faces:
                    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
                    predication=DeepFace.analyze(frame[y:y+h,x:x+w],actions = ['emotion'],enforce_detection=False)
                    emotion_dict[round(time*100)]=predication['emotion']
                    
                    Commen_emotion.append(predication['dominant_emotion'])

                    cv2.putText(frame,str(predication['dominant_emotion'])+ ": " + str(round(predication['emotion'][predication['dominant_emotion']],2)),(x,y-10),font,1,(0,255,0),2, cv2.LINE_AA)
                    break
                try:
                    if(count_time>5):
                        if(mode(Commen_emotion[-10:])!=mode(Commen_emotion[-20:-10])):
                            second=timedelta(seconds=time)
                            empty_dict[mode(Commen_emotion[-20:-10])].append(str(now)+"-"+str(second))
                            #time_list.append([now,second,mode(Commen_emotion[-20:-10])])
                            #print(mode(Commen_emotion[-10:]))
                            #print(mode(Commen_emotion[-20:-10]))
                            count_time=0
                            #print(now,second)
                            
                            now=second
                except Exception as e:
                    #print(e)
                    pass

                count_time=count_time+1
                self.change_pixmap_signal.emit(frame)
            else:
                empty_dict[mode(Commen_emotion[-10:])].append(str(now)+"-"+str(timedelta(seconds=time)))
                Markemotions(empty_dict)
                plot_emotions(emotion_dict)
                plot_piechart(Commen_emotion)
                break
class App(QWidget):
    def __init__(self):
        super().__init__()

        #self.setWindowTitle("PyQt5 Media Player")
        self.setGeometry(350, 100, 700, 500)
        self.setWindowIcon(QIcon('FE.png'))
        
        self.setWindowTitle("Emotion Recognition")
        self.disply_width = 640
        self.display_height = 480
        # create the label that holds the image
        self.image_label = QLabel(self)
        self.image_label.resize(self.disply_width, self.display_height)
        # create a text label
        self.textLabel = QLabel('')

        # create a vertical box layout and add the two labels
        vbox = QVBoxLayout()
        vbox.addWidget(self.image_label)
        vbox.addWidget(self.textLabel)
        # set the vbox layout as the widgets layout
        self.setLayout(vbox)
        
        self.openBtn = QPushButton('Open Video')
        self.openBtn.clicked.connect(self.open_file)
        vbox.addWidget(self.openBtn)
        # create the video capture thread
        self.thread = VideoThread()
        # connect its signal to the update_image slot
        self.thread.change_pixmap_signal.connect(self.update_image)
        # start the thread
        



    @pyqtSlot(np.ndarray)
    def update_image(self, cv_img):
        """Updates the image_label with a new opencv image"""
        qt_img = self.convert_cv_qt(cv_img)
        self.image_label.setPixmap(qt_img)

    def open_file(self):
        global movie_name
        movie_name , _ = QFileDialog.getOpenFileName(self, "Open Video")
        self.thread.start()

    def convert_cv_qt(self, cv_img):
        """Convert from an opencv image to QPixmap"""
        rgb_image = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_image.shape
        bytes_per_line = ch * w
        convert_to_Qt_format = QtGui.QImage(rgb_image.data, w, h, bytes_per_line, QtGui.QImage.Format_RGB888)
        p = convert_to_Qt_format.scaled(self.disply_width, self.display_height, Qt.KeepAspectRatio)
        return QPixmap.fromImage(p)
    
if __name__=="__main__":
    app = QApplication(sys.argv)
    app.setStyle("Fusion")
    palette = QPalette()
    palette.setColor(QPalette.Window, QColor(53, 53, 53))
    palette.setColor(QPalette.WindowText, QColor(255, 255, 255))
    palette.setColor(QPalette.Base, QColor(25, 25, 25))
    palette.setColor(QPalette.AlternateBase, QColor(53, 53, 53))
    palette.setColor(QPalette.ToolTipBase, QColor(255, 255, 255))
    palette.setColor(QPalette.ToolTipText, QColor(255, 255, 255))
    palette.setColor(QPalette.Text, QColor(255, 255, 255))
    palette.setColor(QPalette.Button, QColor(53, 53, 53))
    palette.setColor(QPalette.ButtonText, QColor(255, 255, 255))
    palette.setColor(QPalette.BrightText, QColor(255, 0, 0))
    palette.setColor(QPalette.Link, QColor(42, 130, 218))
    palette.setColor(QPalette.Highlight, QColor(42, 130, 218))
    palette.setColor(QPalette.HighlightedText, QColor(0, 0, 0))
    app.setPalette(palette)
    a = App()
    a.show()
    sys.exit(app.exec_())

SystemExit: 0